<a href="https://colab.research.google.com/github/BeforeTheStorm202/DACN/blob/main/InSDN_hsj_MLP_SHAP_detection_bin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.1/547.1 kB 29.1 MB/s eta 0:00:00


In [2]:
import shap
import pandas as pd
import tensorflow as tf
from tensorflow import keras

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Load trainset

In [3]:
df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/binary/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 77 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

In [4]:
df = df.drop(columns=['Label'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275111 entries, 0 to 275110
Data columns (total 76 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Flow Duration      275111 non-null  float64
 1   Tot Fwd Pkts       275111 non-null  float64
 2   Tot Bwd Pkts       275111 non-null  float64
 3   TotLen Fwd Pkts    275111 non-null  float64
 4   TotLen Bwd Pkts    275111 non-null  float64
 5   Fwd Pkt Len Max    275111 non-null  float64
 6   Fwd Pkt Len Min    275111 non-null  float64
 7   Fwd Pkt Len Mean   275111 non-null  float64
 8   Fwd Pkt Len Std    275111 non-null  float64
 9   Bwd Pkt Len Max    275111 non-null  float64
 10  Bwd Pkt Len Min    275111 non-null  float64
 11  Bwd Pkt Len Mean   275111 non-null  float64
 12  Bwd Pkt Len Std    275111 non-null  float64
 13  Flow Byts/s        275111 non-null  float64
 14  Flow Pkts/s        275111 non-null  float64
 15  Flow IAT Mean      275111 non-null  float64
 16  Fl

# Load adv samples

In [15]:
feature_list = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts',
       'TotLen Bwd Pkts', 'Fwd Pkt Len Max', 'Fwd Pkt Len Min',
       'Fwd Pkt Len Mean', 'Fwd Pkt Len Std', 'Bwd Pkt Len Max',
       'Bwd Pkt Len Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Tot', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Fwd Byts/b Avg', 'Fwd Pkts/b Avg', 'Fwd Blk Rate Avg',
       'Bwd Byts/b Avg', 'Bwd Pkts/b Avg', 'Bwd Blk Rate Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

In [5]:
hsj_path = '/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/adv/24_feature/hsj/hsj_bin_feature'

In [6]:
hsj_df = pd.read_csv(hsj_path)

In [7]:
#hsj_df.columns = feature_list
hsj_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55151 entries, 0 to 55150
Data columns (total 76 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Flow Duration      55151 non-null  float64
 1   Tot Fwd Pkts       55151 non-null  float64
 2   Tot Bwd Pkts       55151 non-null  float64
 3   TotLen Fwd Pkts    55151 non-null  float64
 4   TotLen Bwd Pkts    55151 non-null  float64
 5   Fwd Pkt Len Max    55151 non-null  float64
 6   Fwd Pkt Len Min    55151 non-null  float64
 7   Fwd Pkt Len Mean   55151 non-null  float64
 8   Fwd Pkt Len Std    55151 non-null  float64
 9   Bwd Pkt Len Max    55151 non-null  float64
 10  Bwd Pkt Len Min    55151 non-null  float64
 11  Bwd Pkt Len Mean   55151 non-null  float64
 12  Bwd Pkt Len Std    55151 non-null  float64
 13  Flow Byts/s        55151 non-null  float64
 14  Flow Pkts/s        55151 non-null  float64
 15  Flow IAT Mean      55151 non-null  float64
 16  Flow IAT Std       551

# Load DL-based IDS model

In [8]:
model_mlp = tf.keras.models.load_model('/content/drive/MyDrive/DACN_XAI_Adv_defense/Dataset/insdn/insdn_mlp_bin.h5', compile=True)

# SHAP

In [9]:
explainer = shap.KernelExplainer(model_mlp, shap.sample(df, 100))

In [10]:
hsj_1000 = hsj_df.head(1000)

In [11]:
shap_values = explainer.shap_values(hsj_1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [12]:
print(shap_values[0])

[[ 0.0125374  -0.00219166 -0.00195191 ...  0.          0.00210421
   0.00251801]
 [ 0.00097669  0.00116165  0.00194681 ... -0.00340335  0.00372628
   0.00790232]
 [ 0.00132904 -0.00076516  0.00055625 ... -0.00169805  0.00448393
   0.00769213]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.01198052  0.          0.00130985 ...  0.00109538  0.00330921
   0.00213014]
 [ 0.          0.00804155  0.00470512 ...  0.         -0.00500601
   0.00702134]]


In [19]:
len(shap_values[0])

1000

In [20]:
type(shap_values[0])

numpy.ndarray

In [13]:
data_df = pd.DataFrame(shap_values[0])

In [14]:
data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,0.012537,-0.002192,-0.001952,0.000000,-0.001711,0.001505,0.000000,0.000000,0.003705,0.000000,...,0.001721,0.0,0.000000,0.002317,0.001216,-0.001856,0.005987,0.000000,0.002104,0.002518
1,0.000977,0.001162,0.001947,0.000000,0.000000,-0.008805,0.001960,-0.003474,-0.010789,-0.001890,...,0.000000,0.0,0.001372,0.000000,0.001718,0.000000,0.002460,-0.003403,0.003726,0.007902
2,0.001329,-0.000765,0.000556,-0.001869,-0.002247,0.001566,-0.001187,0.000640,0.002260,0.000429,...,0.001153,0.0,0.000000,0.000420,-0.002892,0.000483,0.001800,-0.001698,0.004484,0.007692
3,0.001954,0.001417,0.000000,-0.001036,0.000000,0.001055,0.001067,0.000932,-0.001585,0.000000,...,0.000000,0.0,0.000918,-0.001564,0.001391,0.000469,0.005257,0.001424,0.005302,0.007083
4,-0.037823,0.000000,0.000000,0.000000,0.000000,-0.006701,0.000000,0.000000,-0.008090,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.047501,0.000000,-0.067582,-0.045446


In [16]:
data_df.to_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_HSJ/1000_samples_[0]_bin.csv', index=False)

In [21]:
data_df.columns = feature_list
data_df.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.012537,-0.002192,-0.001952,0.000000,-0.001711,0.001505,0.000000,0.000000,0.003705,0.000000,...,0.001721,0.0,0.000000,0.002317,0.001216,-0.001856,0.005987,0.000000,0.002104,0.002518
1,0.000977,0.001162,0.001947,0.000000,0.000000,-0.008805,0.001960,-0.003474,-0.010789,-0.001890,...,0.000000,0.0,0.001372,0.000000,0.001718,0.000000,0.002460,-0.003403,0.003726,0.007902
2,0.001329,-0.000765,0.000556,-0.001869,-0.002247,0.001566,-0.001187,0.000640,0.002260,0.000429,...,0.001153,0.0,0.000000,0.000420,-0.002892,0.000483,0.001800,-0.001698,0.004484,0.007692
3,0.001954,0.001417,0.000000,-0.001036,0.000000,0.001055,0.001067,0.000932,-0.001585,0.000000,...,0.000000,0.0,0.000918,-0.001564,0.001391,0.000469,0.005257,0.001424,0.005302,0.007083
4,-0.037823,0.000000,0.000000,0.000000,0.000000,-0.006701,0.000000,0.000000,-0.008090,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.047501,0.000000,-0.067582,-0.045446


In [22]:
data_dict = {}
for idx in range(0, 1000):
  # Select the desired row and get the top 10 values's column names
  row_values = data_df.iloc[idx].nlargest(10).index.tolist()

  # Create a dictionary and add the list with filename as the key
  data_dict['Adv sample ' + str(idx)] = row_values

In [23]:
print(data_dict)

{'Adv sample 0': ['Pkt Len Min', 'Init Bwd Win Byts', 'Down/Up Ratio', 'Flow Duration', 'Bwd Pkt Len Min', 'Flow IAT Max', 'Fwd IAT Max', 'Flow IAT Std', 'Pkt Len Std', 'Pkt Len Max'], 'Adv sample 1': ['SYN Flag Cnt', 'Init Bwd Win Byts', 'Pkt Len Min', 'Idle Min', 'Bwd Pkt Len Min', 'Idle Max', 'Flow IAT Max', 'Fwd IAT Tot', 'Fwd IAT Max', 'Fwd Pkts/s'], 'Adv sample 2': ['SYN Flag Cnt', 'Init Bwd Win Byts', 'Pkt Len Min', 'Bwd Pkt Len Min', 'Idle Min', 'Down/Up Ratio', 'Flow IAT Max', 'Idle Max', 'Pkt Len Std', 'Fwd IAT Tot'], 'Adv sample 3': ['SYN Flag Cnt', 'Init Bwd Win Byts', 'Pkt Len Min', 'Down/Up Ratio', 'Idle Min', 'Bwd Pkt Len Min', 'Idle Max', 'Idle Mean', 'Pkt Len Std', 'Pkt Len Mean'], 'Adv sample 4': ['ACK Flag Cnt', 'Bwd IAT Std', 'Init Bwd Win Byts', 'Bwd IAT Max', 'Bwd IAT Tot', 'Flow IAT Mean', 'Pkt Len Min', 'Fwd IAT Mean', 'Bwd URG Flags', 'Bwd IAT Min'], 'Adv sample 5': ['Bwd Pkts/s', 'Init Bwd Win Byts', 'Pkt Len Min', 'Down/Up Ratio', 'Bwd Pkt Len Min', 'Flow IAT

In [24]:
len(shap_values[0])

1000

In [25]:
len(data_dict)

1000

# Detection phase

In [26]:
import os
import pandas as pd

In [27]:
whitelist_df = pd.read_csv('/content/drive/MyDrive/DACN_XAI_Adv_defense/InSDN/MLP_top24_InSDN_bin.csv')
whitelist_df.head(24)

,Feature,Frequency
0,Init Bwd Win Byts,684
1,Idle Max,604
2,Bwd Pkt Len Max,571
3,Idle Mean,535
4,Idle Min,535
5,Bwd Pkt Len Mean,462
6,Bwd Pkt Len Std,444
7,Pkt Len Min,383
8,Tot Fwd Pkts,364
9,Bwd Seg Size Avg,358


## Detection rate (top 24) = 969/1000

In [28]:
whitelist = whitelist_df.iloc[:, 0][:24].tolist()
print(whitelist)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std']


In [29]:
alert = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist:
        alert.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert)))

Alert list length:983


## Detection rate (top 38) = 783/1000

In [30]:
whitelist2 = whitelist_df.iloc[:, 0][:38].tolist()
print(whitelist2)

['Init Bwd Win Byts', 'Idle Max', 'Bwd Pkt Len Max', 'Idle Mean', 'Idle Min', 'Bwd Pkt Len Mean', 'Bwd Pkt Len Std', 'Pkt Len Min', 'Tot Fwd Pkts', 'Bwd Seg Size Avg', 'Bwd IAT Max', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Bwd IAT Tot', 'SYN Flag Cnt', 'Flow Pkts/s', 'Bwd IAT Std', 'Bwd Pkt Len Min', 'Fwd Pkt Len Min', 'Flow Duration', 'Down/Up Ratio', 'Flow IAT Min', 'Fwd Pkt Len Std', 'Fwd Pkts/s', 'Active Min', 'Fwd IAT Std', 'Flow IAT Mean', 'Fwd IAT Mean', 'Bwd Header Len', 'Pkt Len Var', 'Active Mean', 'Fwd IAT Tot', 'Active Max', 'Idle Std', 'Active Std', 'Flow IAT Std', 'Subflow Bwd Pkts']


In [31]:
alert2 = []

# Iterate over each key-value pair in the data_dict
for key, value in data_dict.items():
    # Check if there are one feature is not in the whitelist
    for string in value:
      if string not in whitelist2:
        alert2.append(key)
        break

# Print the alert dictionary
print("Alert list length:" + str(len(alert2)))

Alert list length:969
